In [ ]:
from __future__ import print_function

import json
import urllib
import uuid
import boto3
import re
# ReadOnce Object From:
# http://stackoverflow.com/questions/30675862/infinite-loop-when-streaming-a-gz-file-from-s3-using-boto
class ReadOnce(object):
    def __init__(self, k):
        self.key = k
        self.has_read_once = False

    def read(self, size=0):
        if self.has_read_once:
            return b”
        data = self.key.read(size)
        if not data:
            self.has_read_once = True
            return data

print(‘Loading IO function’)

s3 = boto3.client(‘s3’)

def lambda_handler(event, context):
print(“Received event: ” + json.dumps(event, indent=2))

# Get the object from the event and show its content type
inbucket = event[‘Records’][0][‘s3’][‘bucket’][‘name’]
outbucket = “outlambda”
inkey = urllib.unquote_plus(event[‘Records’][0][‘s3’][‘object’][‘key’].encode(‘utf8’))
outkey = “out” + inkey
try:
infile = s3.get_object(Bucket=inbucket, Key=inkey)

except Exception as e:
print(e)
print(‘Error getting object {} from bucket {}. Make sure they exist and your bucket is in the same region as this function.’.format(inkey, bucket))
raise e

inbody = infile[‘Body’]
tmp_path = ‘/tmp/{}{}’.format(uuid.uuid4(), “tmp.txt”)
# upload_path = ‘/tmp/resized-{}’.format(key)

with open(tmp_path,’w’) as out:
unfinished_line = ”
bytes=inbody.read(4096)
while( bytes ):
bytes = unfinished_line + bytes
#split on whatever, or use a regex with re.split()
lines = bytes.split(‘\n’)
print (“bytes %s” % bytes)
unfinished_line = lines.pop()
for line in lines:
print (“line %s” % line)
out.write(line)
# yield line
bytes=inbody.read(4096)
if(unfinished_line):
out.write(unfinished_line)
#
# Upload the file to S3
#
tmp = open(tmp_path,”r”)
try:
outfile = s3.put_object(Bucket=outbucket,Key=outkey,Body=tmp)
except Exception as e:
print(e)
print(‘Error putting object {} from bucket {} Body {}. Make sure they exist and your bucket is in the same region as this function.’.format(outkey, outbucket,”tmp.txt”))
raise e

tmp.close()